In [32]:
import requests
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from datetime import datetime, timedelta
import yfinance as yf
import alpaca_trade_api as tradeapi
import time
from apscheduler.schedulers.background import BackgroundScheduler

In [33]:
# Alpaca alpaca setup
FMP_API_KEY = 'vYT1P4526Gyrp3ME9sLvJMiT7EaINpgN'
ALPACA_API_KEY = 'PKMKEW5LUNRY6HISJNDM'
ALPACA_SECRET_KEY = 'dGvvZvzIAbyoOFTer4tAa9WlMCSBOghXm9L9EEF3'
BASE_URL='https://paper-api.alpaca.markets'
alpaca = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, base_url=BASE_URL)

In [34]:
def get_nasdaq100_symbols():
    url = 'https://en.wikipedia.org/wiki/NASDAQ-100'
    html = requests.get(url).text
    df = pd.read_html(html)[4]  # The NASDAQ-100 table is the fourth table on the page
    return df['Ticker'].tolist()

# Step 1: Fetch Nasdaq 100 symbols
nasdaq_symbols = get_nasdaq100_symbols()


/tmp/ipykernel_26543/4161348169.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[4]  # The NASDAQ-100 table is the fourth table on the page


In [35]:
def get_dowjones_symbols():
    url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'
    html = requests.get(url).text
    df = pd.read_html(html)[1]  # The Dow Jones table is the second table on the page
    return df['Symbol'].tolist()

# Step 1: Fetch Dow Jones symbols
dow_jones_symbols = get_dowjones_symbols()


/tmp/ipykernel_26543/3419010943.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[1]  # The Dow Jones table is the second table on the page


In [36]:
# Step 1: Create dataframes for each index
nasdaq_df = pd.DataFrame({'symbol': nasdaq_symbols, 'index': 'Nasdaq 100'})
dow_jones_df = pd.DataFrame({'symbol': dow_jones_symbols, 'index': 'Dow Jones'})

# Step 2: Combine the dataframes
combined_df = pd.concat([nasdaq_df, dow_jones_df], ignore_index=True)

In [37]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load both FinBERT models
model_name_prosus = "ProsusAI/finbert"
tokenizer_prosus = AutoTokenizer.from_pretrained(model_name_prosus)
model_prosus = AutoModelForSequenceClassification.from_pretrained(model_name_prosus)
sentiment_analyzer_prosus = pipeline("sentiment-analysis", model=model_prosus, tokenizer=tokenizer_prosus)

model_name_yiyang = "yiyanghkust/finbert-tone"
tokenizer_yiyang = AutoTokenizer.from_pretrained(model_name_yiyang)
model_yiyang = AutoModelForSequenceClassification.from_pretrained(model_name_yiyang)
sentiment_analyzer_yiyang = pipeline("sentiment-analysis", model=model_yiyang, tokenizer=tokenizer_yiyang)

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [38]:
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import requests
import yfinance as yf
import pandas as pd

# Function to fetch Alpaca news
def get_alpaca_news(symbol=None, limit=50):
    url = f"https://data.alpaca.markets/v1beta1/news?sort=desc&limit={limit}"
    if symbol:
        url += f"&symbols={symbol}"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": ALPACA_API_KEY,
        "APCA-API-SECRET-KEY": ALPACA_SECRET_KEY
    }
    response = requests.get(url, headers=headers)
    return response.json().get('news', [])

# Function to fetch Yahoo Finance news (headlines only)
def fetch_yfinance_news(symbol):
    ticker = yf.Ticker(symbol)
    news_items = ticker.news
    news_data = []
    for article in news_items:
        news_data.append({
            'title': article.get('title', 'No title'),
            'publisher': article.get('publisher', 'Unknown publisher'),
            'link': article.get('link', ''),
            'date': article.get('providerPublishTime', None)  # Use None if date is missing
        })
    return news_data

# Function to filter articles within the date range
def filter_articles_by_date(news_items, start_date, end_date):
    filtered_news = []
    for article in news_items:
        article_date = article.get('date')
        if article_date:
            article_date = datetime.fromtimestamp(article_date) if isinstance(article_date, (int, float)) else datetime.strptime(article_date, '%Y-%m-%dT%H:%M:%S.%fZ')
            if start_date <= article_date <= end_date:
                filtered_news.append(article)
    return filtered_news

# Function to fetch news data and perform sentiment analysis using both FinBERT models
def fetch_news_and_analyze(row, start_date, end_date):
    symbol = row['symbol']
    index_name = row['index']
    news_data = []
    print(f"Fetching news for {symbol} ({index_name})...")
    
    # Fetch Alpaca news
    alpaca_articles = get_alpaca_news(symbol)
    filtered_alpaca_articles = filter_articles_by_date(alpaca_articles, start_date, end_date)
    for article in filtered_alpaca_articles:
        headline = article['headline']
        
        # Perform sentiment analysis using both models
        sentiment_prosus = sentiment_analyzer_prosus(headline)[0]
        sentiment_yiyang = sentiment_analyzer_yiyang(headline)[0]
        
        news_data.append({
            'symbol': symbol,
            'index': index_name,
            'source': 'Alpaca',
            'headline': headline,
            'sentiment_prosus': sentiment_prosus['label'],
            'confidence_prosus': sentiment_prosus['score'],
            'sentiment_yiyang': sentiment_yiyang['label'],
            'confidence_yiyang': sentiment_yiyang['score'],
            'date': article['created_at']
        })
    
    # Fetch Yahoo Finance news
    yfinance_articles = fetch_yfinance_news(symbol)
    yfinance_articles_filtered = filter_articles_by_date(yfinance_articles, start_date, end_date)
    for article in yfinance_articles_filtered:
        headline = article['title']
        
        # Perform sentiment analysis using both models
        sentiment_prosus = sentiment_analyzer_prosus(headline)[0]
        sentiment_yiyang = sentiment_analyzer_yiyang(headline)[0]
        
        news_data.append({
            'symbol': symbol,
            'index': index_name,
            'source': 'Yahoo Finance',
            'headline': headline,
            'sentiment_prosus': sentiment_prosus['label'],
            'confidence_prosus': sentiment_prosus['score'],
            'sentiment_yiyang': sentiment_yiyang['label'],
            'confidence_yiyang': sentiment_yiyang['score'],
            'date': datetime.fromtimestamp(article['date']).strftime('%Y-%m-%d %H:%M:%S') if article['date'] else 'Unknown'
        })
    
    return news_data

# Gather news data using ThreadPoolExecutor for parallel processing
def gather_news_data_parallel(combined_df, start_date, end_date):
    news_data = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(fetch_news_and_analyze, [row for _, row in combined_df.iterrows()], [start_date]*len(combined_df), [end_date]*len(combined_df))
        for result in results:
            news_data.extend(result)
    return pd.DataFrame(news_data)


In [39]:
# Define your date range
start_date = datetime.now() - pd.DateOffset(days=1)  # Last 1 day
end_date = datetime.now()

# Example usage
news_df = gather_news_data_parallel(combined_df, start_date, end_date)

# Display the DataFrame with sentiment analysis
print(news_df)

Fetching news for ADBE (Nasdaq 100)...
Fetching news for ABNB (Nasdaq 100)...
Fetching news for GOOGL (Nasdaq 100)...
Fetching news for GOOG (Nasdaq 100)...
Fetching news for AMZN (Nasdaq 100)...
Fetching news for AMD (Nasdaq 100)...
Fetching news for AEP (Nasdaq 100)...
Fetching news for AMGN (Nasdaq 100)...
Fetching news for ADI (Nasdaq 100)...
Fetching news for ANSS (Nasdaq 100)...
Fetching news for AAPL (Nasdaq 100)...
Fetching news for AMAT (Nasdaq 100)...
Fetching news for ARM (Nasdaq 100)...
Fetching news for ASML (Nasdaq 100)...
Fetching news for AZN (Nasdaq 100)...
Fetching news for TEAM (Nasdaq 100)...
Fetching news for ADSK (Nasdaq 100)...
Fetching news for ADP (Nasdaq 100)...
Fetching news for BKR (Nasdaq 100)...
Fetching news for BIIB (Nasdaq 100)...
Fetching news for BKNG (Nasdaq 100)...
Fetching news for AVGO (Nasdaq 100)...
Fetching news for CDNS (Nasdaq 100)...
Fetching news for CDW (Nasdaq 100)...
Fetching news for CHTR (Nasdaq 100)...
Fetching news for CTAS (Nasdaq 1

In [40]:
def map_sentiment_to_numeric(sentiment_label):
    sentiment_map = {'positive': 1, 'neutral': 0, 'negative': -1}
    return sentiment_map.get(sentiment_label.lower(), 0)  # Default to 0 if the label is not recognized

def calculate_weighted_sentiment_score(news_data):
    # Convert sentiment labels to numeric values
    news_data['numeric_prosus'] = news_data['sentiment_prosus'].apply(map_sentiment_to_numeric)
    news_data['numeric_yiyang'] = news_data['sentiment_yiyang'].apply(map_sentiment_to_numeric)
    
    # Calculate weighted sentiment scores by multiplying numeric sentiment by confidence
    news_data['weighted_prosus'] = news_data['numeric_prosus'] * news_data['confidence_prosus']
    news_data['weighted_yiyang'] = news_data['numeric_yiyang'] * news_data['confidence_yiyang']
    
    return news_data


In [41]:
def aggregate_sentiment_scores(news_data):
    # Calculate the weighted sentiment score for each company
    weighted_scores = calculate_weighted_sentiment_score(news_data)
    
    # Aggregate the weighted scores to get the overall sentiment score for each company
    sentiment_scores = weighted_scores.groupby('symbol').agg({
        'weighted_prosus': 'mean',
        'weighted_yiyang': 'mean'
    }).reset_index()
    
    # Calculate the overall sentiment score as an average of the two weighted scores
    sentiment_scores['Overall_Sentiment_Score'] = sentiment_scores[['weighted_prosus', 'weighted_yiyang']].mean(axis=1)
    
    return sentiment_scores[['symbol', 'Overall_Sentiment_Score']]


In [42]:
# Assuming `news_data` contains your sentiment analysis results
sentiment_scores = aggregate_sentiment_scores(news_df)

# Display the combined sentiment scores
print(sentiment_scores)


   symbol  Overall_Sentiment_Score
0    AAPL                -0.073178
1    ABNB                 0.568629
2    ADBE                -0.177852
3     ADI                 0.357166
4     AMD                 0.386937
..    ...                      ...
88   VRTX                 0.000000
89     VZ                 0.450756
90    WBD                 0.000000
91   WDAY                 0.467778
92    WMT                 0.395396

[93 rows x 2 columns]


In [43]:
# Import TimeFrame from the Alpaca trade API
from alpaca_trade_api.rest import TimeFrame

# Function to fetch historical data using Alpaca
def fetch_historical_data_alpaca(symbols, start_date, end_date):
    historical_data = []
    for symbol in symbols:
        data = alpaca.get_bars(symbol, TimeFrame.Day, start=start_date, end=end_date).df
        data['symbol'] = symbol  # Add symbol column to distinguish data
        historical_data.append(data)
    return pd.concat(historical_data)

# Fetch historical data from Alpaca for selected companies
start_date = "2020-01-01"
end_date = "2024-07-31"
selected_companies = combined_df['symbol'].unique().tolist()
historical_data = fetch_historical_data_alpaca(selected_companies, start_date, end_date)

# Now you can use historical_data for your technical analysis


In [44]:
import numpy as np
import pandas as pd

# Technical Analysis Functions
def calculate_sma(data, window=10):
    return data['close'].rolling(window=window).mean()

def generate_sma_crossover_signals(data, short_window=5, long_window=10):
    data['SMA_Short'] = calculate_sma(data, short_window)
    data['SMA_Long'] = calculate_sma(data, long_window)
    data['SMA_Crossover_Signal'] = np.where(data['SMA_Short'] > data['SMA_Long'], 1, -1)
    return data

def calculate_rsi(data, window=7):
    delta = data['close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def generate_rsi_signals(data, window=7):
    data['RSI_14'] = calculate_rsi(data)
    data['RSI_Signal'] = np.where(data['RSI_14'] < 30, 1, 
                                  np.where(data['RSI_14'] > 70, -1, 0))
    return data

def calculate_macd(data):
    ema_6 = data['close'].ewm(span=6, adjust=False).mean()
    ema_13 = data['close'].ewm(span=13, adjust=False).mean()
    macd = ema_6 - ema_13
    signal = macd.ewm(span=5, adjust=False).mean()
    return macd, signal

def generate_macd_signals(data):
    data['MACD'], data['MACD_Signal'] = calculate_macd(data)
    data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']
    data['MACD_Signal_Strength'] = np.where(data['MACD_Histogram'] > 0, 1, 0)
    return data

# EMA Calculation and Signal
def calculate_ema(data, window=10):
    return data['close'].ewm(span=window, adjust=False).mean()

def generate_ema_signals(data, window=10):
    data[f'EMA_{window}'] = calculate_ema(data, window)
    data[f'EMA_Signal_{window}'] = np.where(data['close'] > data[f'EMA_{window}'], 1, -1)
    return data

# Bollinger Bands Calculation and Signal
def calculate_bollinger_bands(data, window=10, num_std_dev=2):
    sma = calculate_sma(data, window)
    rolling_std = data['close'].rolling(window=window).std()
    data['Bollinger_Upper'] = sma + (rolling_std * num_std_dev)
    data['Bollinger_Lower'] = sma - (rolling_std * num_std_dev)
    return data

def generate_bollinger_signals(data):
    data = calculate_bollinger_bands(data, window=10)  # Use the window argument here
    data['Bollinger_Signal'] = np.where(data['close'] < data['Bollinger_Lower'], 1,
                                        np.where(data['close'] > data['Bollinger_Upper'], -1, 0))
    return data

# Stochastic Oscillator Calculation and Signal
def calculate_stochastic_oscillator(data, window=7):
    low_min = data['low'].rolling(window=window).min()
    high_max = data['high'].rolling(window=window).max()
    stoch = 100 * (data['close'] - low_min) / (high_max - low_min)
    return stoch

def generate_stochastic_signals(data, window=7):
    data['Stochastic_%K'] = calculate_stochastic_oscillator(data, window)
    data['Stochastic_Signal'] = np.where(data['Stochastic_%K'] < 20, 1,
                                         np.where(data['Stochastic_%K'] > 80, -1, 0))
    return data

# Function to perform technical analysis
def perform_technical_analysis(historical_data):
    results = []

    for symbol in historical_data['symbol'].unique():
        stock_data = historical_data[historical_data['symbol'] == symbol].copy()

        # Generate signals for each technical indicator
        stock_data = generate_sma_crossover_signals(stock_data)
        stock_data = generate_rsi_signals(stock_data)
        stock_data = generate_macd_signals(stock_data)
        stock_data = generate_ema_signals(stock_data, window=10)  # Shorter EMA
        stock_data = generate_bollinger_signals(stock_data)  # Shorter Bollinger Band
        stock_data = generate_stochastic_signals(stock_data, window=7)  # Shorter Stochastic Oscillator

        # Create a summary of the latest signals for this stock
        signals = {
            'symbol': symbol,
            'SMA_Signal': stock_data['SMA_Crossover_Signal'].iloc[-1],
            'RSI_Signal': stock_data['RSI_Signal'].iloc[-1],
            'MACD_Signal': stock_data['MACD_Signal_Strength'].iloc[-1],
            'EMA_Signal': stock_data[f'EMA_Signal_10'].iloc[-1],
            'Bollinger_Signal': stock_data['Bollinger_Signal'].iloc[-1],
            'Stochastic_Signal': stock_data['Stochastic_Signal'].iloc[-1]
        }

        # More aggressive decision-making
        if (signals['SMA_Signal'] == 1 and
            signals['MACD_Signal'] == 1 and
            signals['RSI_Signal'] >= 0 and
            signals['Bollinger_Signal'] >= 0 and
            signals['Stochastic_Signal'] >= 0):
            signals['Final_Decision'] = 'Buy'
        elif (signals['SMA_Signal'] == -1 or
              signals['MACD_Signal'] == 0 or
              signals['RSI_Signal'] == -1 or
              signals['Bollinger_Signal'] < 0 or
              signals['Stochastic_Signal'] == -1):
            signals['Final_Decision'] = 'Sell'
        else:
            signals['Final_Decision'] = 'Hold'

        results.append(signals)

    # Convert results to a DataFrame
    signals_df = pd.DataFrame(results)
    return signals_df

# Example usage
signals_df = perform_technical_analysis(historical_data)
print(signals_df)


    symbol  SMA_Signal  RSI_Signal  MACD_Signal  EMA_Signal  Bollinger_Signal  \
0     ADBE          -1           0            1           1                 0   
1     ABNB          -1           1            0          -1                 0   
2    GOOGL          -1           1            1          -1                 0   
3     GOOG          -1           1            1          -1                 0   
4     AMZN          -1           0            1           1                 0   
..     ...         ...         ...          ...         ...               ...   
119    TRV           1          -1            1           1                 0   
120    UNH           1          -1            0           1                 0   
121     VZ          -1          -1            1           1                 0   
122      V          -1           0            1           1                 0   
123    WMT          -1           1            0          -1                 0   

     Stochastic_Signal Fina

In [45]:
# Perform a left join to add the overall_sentiment_score to the signals_df
signals_df = signals_df.merge(sentiment_scores[['symbol', 'Overall_Sentiment_Score']], on='symbol', how='left')

In [46]:
# Fill missing sentiment scores with 0 (neutral sentiment)
signals_df['Overall_Sentiment_Score'].fillna(0, inplace=True)

# Rename the 'Final_Decision' column to 'Technical_Analysis_Results'
signals_df.rename(columns={'Final_Decision': 'Technical_Analysis_Results'}, inplace=True)

# Display the combined DataFrame
print(signals_df)

    symbol  SMA_Signal  RSI_Signal  MACD_Signal  EMA_Signal  Bollinger_Signal  \
0     ADBE          -1           0            1           1                 0   
1     ABNB          -1           1            0          -1                 0   
2    GOOGL          -1           1            1          -1                 0   
3     GOOG          -1           1            1          -1                 0   
4     AMZN          -1           0            1           1                 0   
..     ...         ...         ...          ...         ...               ...   
119    TRV           1          -1            1           1                 0   
120    UNH           1          -1            0           1                 0   
121     VZ          -1          -1            1           1                 0   
122      V          -1           0            1           1                 0   
123    WMT          -1           1            0          -1                 0   

     Stochastic_Signal Tech

In [47]:
# Define the rules for combining sentiment and technical analysis results
def categorize_final_decision(row):
    sentiment = row['Overall_Sentiment_Score']
    technical = row['Technical_Analysis_Results']

    # Scenario when technical analysis says Sell, but sentiment is high
    if technical == 'Sell' and sentiment > 0.5:
        # Check other indicators, like RSI_Signal, MACD_Signal, and Bollinger_Signal
        if (row['RSI_Signal'] > 0) or (row['MACD_Signal'] > 0) or (row['Bollinger_Signal'] >= 0):
            return 'Buy'
        else:
            return 'No Action'
    
    # Adjusted sentiment thresholds
    if technical == 'Buy' and sentiment >= 0.4:
        return 'Strong Buy'
    elif technical == 'Buy' and sentiment >= 0:
        return 'Buy'
    elif technical == 'Sell' and sentiment < -0.1:
        return 'Strong Sell'
    elif technical == 'Sell' and sentiment < 0:
        return 'Sell'
    else:
        return 'No Action'

# Apply the function to create the final decision column
signals_df['Final_Decision'] = signals_df.apply(categorize_final_decision, axis=1)

# Display the updated DataFrame with the new Final_Decision column
print(signals_df)


    symbol  SMA_Signal  RSI_Signal  MACD_Signal  EMA_Signal  Bollinger_Signal  \
0     ADBE          -1           0            1           1                 0   
1     ABNB          -1           1            0          -1                 0   
2    GOOGL          -1           1            1          -1                 0   
3     GOOG          -1           1            1          -1                 0   
4     AMZN          -1           0            1           1                 0   
..     ...         ...         ...          ...         ...               ...   
119    TRV           1          -1            1           1                 0   
120    UNH           1          -1            0           1                 0   
121     VZ          -1          -1            1           1                 0   
122      V          -1           0            1           1                 0   
123    WMT          -1           1            0          -1                 0   

     Stochastic_Signal Tech

# Execution of the trading strategy

In [55]:
# Function to retrieve current positions (your portfolio)
def get_portfolio():
    positions = alpaca.list_positions()
    portfolio = {}
    for position in positions:
        symbol = position.symbol
        qty = int(position.qty)  # Convert quantity to an integer
        print(f"Symbol: {symbol}, Quantity: {qty}, Type: {type(qty)}")  # Confirm the conversion
        avg_price = float(position.avg_entry_price)
        current_price = float(position.current_price)
        portfolio[symbol] = {
            'quantity': qty,
            'average_price': avg_price,
            'current_price': current_price
        }
    return portfolio



# Function to extract symbols from the portfolio dictionary
def get_portfolio_symbols(portfolio):
    return list(portfolio.keys())


In [56]:
import requests

# Fetch latest stock quotes from FMP
def fetch_stock_quotes(symbols, api_key):
    url = f'https://financialmodelingprep.com/api/v3/quote/{",".join(symbols)}?apikey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching stock quotes: {response.status_code}")
        return []

In [57]:
# Function to make trading decisions
def make_trade_decision(symbol, action, qty):
    try:
        if action == 'buy' and qty > 0:
            alpaca.submit_order(
                symbol=symbol,
                qty=qty,
                side='buy',
                type='market',
                time_in_force='gtc'
            )
            print(f"Buying {qty} shares of {symbol}")
        elif action == 'sell' and qty > 0:
            alpaca.submit_order(
                symbol=symbol,
                qty=qty,
                side='sell',
                type='market',
                time_in_force='gtc'
            )
            print(f"Selling {qty} shares of {symbol}")
    except Exception as e:
        print(f"Error placing order: {e}")

# Function to check if you own a stock
def check_if_own_stock(symbol, portfolio):
    # Return the stock details if it exists in the portfolio, else return None
    return portfolio.get(symbol)


In [58]:
def execute_strategy(signals_df, portfolio, total_daily_budget=20000):
    symbols = signals_df['symbol'].tolist()
    quotes = fetch_stock_quotes(symbols, FMP_API_KEY)
    
    if not quotes:
        print("No quotes fetched, skipping trade execution.")
        return
    
    # Filter for stocks with a buy or strong buy signal
    buy_signals = signals_df[signals_df['Final_Decision'].isin(['Buy', 'Strong Buy'])]
    # Filter for stocks with a sell or strong sell signal
    sell_signals = signals_df[signals_df['Final_Decision'].isin(['Sell', 'Strong Sell'])]

    if buy_signals.empty and sell_signals.empty:
        print("No buy or sell signals today.")
        return

    # Divide the daily budget equally among all buy signals
    if not buy_signals.empty:
        individual_budget = total_daily_budget / len(buy_signals)
    else:
        individual_budget = 0

    # Process buy signals
    for i, row in buy_signals.iterrows():
        symbol = row['symbol']
        final_decision = row['Final_Decision']
        
        # Get the latest price
        quote = next((q for q in quotes if q['symbol'] == symbol), None)
        if quote is None:
            print(f"No quote available for {symbol}, skipping.")
            continue
        
        price = quote['price']
        qty = int(individual_budget / price)  # Calculate quantity based on the allocated budget

        if qty <= 0:
            print(f"Not enough budget to buy even one share of {symbol}. Skipping.")
            continue

        # Check if the stock is already in the portfolio
        own_stock = check_if_own_stock(symbol, portfolio)

        if isinstance(own_stock, dict):  # Ensure own_stock is a dictionary
            # If you already own the stock
            if final_decision == 'Strong Buy':
                print(f"Already own {symbol}. Consider buying more.")
                make_trade_decision(symbol, 'buy', qty)
        else:
            # If you do not own the stock
            if final_decision == 'Buy' or final_decision == 'Strong Buy':
                print(f"{final_decision} signal for {symbol}. Buying it.")
                make_trade_decision(symbol, 'buy', qty)

    # Process sell signals
    for i, row in sell_signals.iterrows():
        symbol = row['symbol']
        final_decision = row['Final_Decision']
        
        # Get the latest price
        quote = next((q for q in quotes if q['symbol'] == symbol), None)
        if quote is None:
            print(f"No quote available for {symbol}, skipping.")
            continue
        
        # Check if the stock is already in the portfolio
        own_stock = check_if_own_stock(symbol, portfolio)

        if isinstance(own_stock, dict):  # Ensure own_stock is a dictionary
            # If you already own the stock
            if final_decision == 'Sell' or final_decision == 'Strong Sell':
                print(f"Selling {symbol} based on {final_decision}.")
                make_trade_decision(symbol, 'sell', own_stock['quantity'])
        else:
            print(f"{symbol} is not in the portfolio, skipping sell signal.")


In [59]:
# Example usage
portfolio = get_portfolio()  # Fetch the portfolio from Alpaca
total_daily_budget = 50  # Set your daily budget to $10,000
execute_strategy(signals_df, portfolio)  # Execute the trading strategy

Symbol: AAPL, Quantity: 8, Type: <class 'int'>
Symbol: ABNB, Quantity: 8, Type: <class 'int'>
Symbol: ADBE, Quantity: 4, Type: <class 'int'>
Symbol: ADSK, Quantity: 2, Type: <class 'int'>
Symbol: AMGN, Quantity: 3, Type: <class 'int'>
Symbol: AXP, Quantity: 6, Type: <class 'int'>
Symbol: BKR, Quantity: 81, Type: <class 'int'>
Symbol: CCEP, Quantity: 8, Type: <class 'int'>
Symbol: CPRT, Quantity: 12, Type: <class 'int'>
Symbol: CRM, Quantity: 15, Type: <class 'int'>
Symbol: CRWD, Quantity: 9, Type: <class 'int'>
Symbol: CSX, Quantity: 31, Type: <class 'int'>
Symbol: CTAS, Quantity: 1, Type: <class 'int'>
Symbol: DOW, Quantity: 19, Type: <class 'int'>
Symbol: DXCM, Quantity: 14, Type: <class 'int'>
Symbol: FANG, Quantity: 10, Type: <class 'int'>
Symbol: GEHC, Quantity: 22, Type: <class 'int'>
Symbol: GILD, Quantity: 32, Type: <class 'int'>
Symbol: IBM, Quantity: 5, Type: <class 'int'>
Symbol: IDXX, Quantity: 1, Type: <class 'int'>
Symbol: INTU, Quantity: 3, Type: <class 'int'>
Symbol: JP


# Day Trading Enhancements

This section includes updates for day trading suitability, enhancing real-time data handling, trading signals, 
risk management, and backtesting capabilities. These changes will enable quick and informed intraday trading decisions.


## Enhanced Trading Signals with Real-Time Data and Sentiment Analysis

In [ ]:

import numpy as np

# Enhanced trading signal function using real-time data and sentiment analysis
def enhanced_trading_signals(technical_signal, sentiment_score):
    if technical_signal == 'buy' and sentiment_score > 0.2:
        return 'strong buy'
    elif technical_signal == 'sell' and sentiment_score < -0.2:
        return 'strong sell'
    elif technical_signal == 'buy':
        return 'buy'
    elif technical_signal == 'sell':
        return 'sell'
    else:
        return 'hold'

# Example usage with mock data
technical_signals = ['buy', 'sell', 'buy', 'hold', 'sell']
sentiment_scores = np.random.uniform(-1, 1, 5)  # Simulated sentiment scores
enhanced_signals = [enhanced_trading_signals(ts, ss) for ts, ss in zip(technical_signals, sentiment_scores)]
print("Enhanced Trading Signals:", enhanced_signals)


## Advanced Risk Management Strategies

In [ ]:

# Risk management strategy incorporating stop-loss and take-profit levels
def risk_management(signal, entry_price, current_price, stop_loss=0.02, take_profit=0.05):
    if signal in ['strong buy', 'buy']:
        # Calculate stop loss and take profit thresholds
        stop_loss_price = entry_price * (1 - stop_loss)
        take_profit_price = entry_price * (1 + take_profit)
        if current_price <= stop_loss_price:
            return 'stop loss'
        elif current_price >= take_profit_price:
            return 'take profit'
        else:
            return 'hold'
    elif signal in ['strong sell', 'sell']:
        return 'execute sell'
    return 'no action'

# Example usage with mock data
entry_price = 100
current_prices = np.random.uniform(95, 105, len(enhanced_signals))  # Simulated current prices
actions = [risk_management(signal, entry_price, cp) for signal, cp in zip(enhanced_signals, current_prices)]
print("Risk Management Actions:", actions)


## Backtesting the Trading Strategy

In [ ]:

# Backtesting function to assess the performance of the strategy
def backtest_trading_strategy(prices, signals):
    cash = 10000  # Starting cash amount
    position = 0  # Number of shares held
    portfolio_value = cash

    for price, signal in zip(prices, signals):
        if signal in ['strong buy', 'buy'] and cash >= price:
            position += 1
            cash -= price
        elif signal in ['strong sell', 'sell'] and position > 0:
            position -= 1
            cash += price

        # Update portfolio value based on current position
        portfolio_value = cash + position * price

    return portfolio_value

# Simulated price data for backtesting
mock_prices = np.random.uniform(100, 150, len(enhanced_signals))
final_value = backtest_trading_strategy(mock_prices, enhanced_signals)
print(f"Final Portfolio Value after Backtesting: ${final_value:.2f}")
